In [121]:
import pandas as pd

In [122]:
df = pd.read_csv("./streetview_parcel_basic.csv")

In [123]:
hsv = pd.read_csv("./GSV/FISP/GSV/60/whole_metadata.csv")

In [124]:
hsv.set_index("filename",drop=True,inplace=True)

In [125]:
df = df.join(hsv,on="filename")

###Removal of blank returns

In [126]:
df.hue.value_counts().head()

0.133593    22606
0.205820     4692
0.279342     3055
0.339471      888
0.354805      750
dtype: int64

In [127]:
import numpy as np

In [128]:
np.mean(df.hue==0.133593)

0.0

In [129]:
df.hue.head(10)

0    0.205820
1    0.205820
2    0.309822
3    0.309822
4    0.205820
5    0.294937
6    0.294937
7    0.133593
8    0.317456
9    0.309822
Name: hue, dtype: float64

In [130]:
df.hue.loc[7]

0.13359333631699999

In [131]:
np.mean(df.hue==0.13359333631699999)

0.059061125103198907

In [132]:
df = df[df.hue!=0.13359333631699999]

###Subsetting

In [133]:
# Need the hoods in here
# But for now, I need to just visualize some of this

In [134]:
df.nucleus_us.value_counts().head()

110    117590
111     83406
171     73594
170     15659
130     11879
dtype: int64

In [135]:
# 110 is 'single-family generic'

In [136]:
sfg = df[df.nucleus_us==110]

In [137]:
np.mean(sfg.asr_total==0)

2.5512373501148056e-05

###Circle plotz

In [138]:
from shapely.geometry import Point
from PIL import Image

In [139]:
thumb_side = 64

In [140]:
DIR = "/Users/damoncrockett/Desktop/FISP/VIZ/"

In [141]:
var = "asr_total"

In [142]:
sample_size = 10000

In [143]:
sfg = sfg.sample(n=sample_size)

In [144]:
n = len(sfg)

In [145]:
sfg.sort(var,ascending=False,inplace=True)

In [146]:
sfg.reset_index(drop=True,inplace=True)

In [147]:
side = int(round(np.sqrt(n))) + 4
canvas = Image.new('RGB',(side * thumb_side, side * thumb_side),(50,50,50))
x,y = range(side) * side, np.repeat(range(side),side)
grid_list = pd.DataFrame(x,columns=['x'])
grid_list['y'] = y
        
point = []
m = len(grid_list.index)
for i in range(m):
    point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))
grid_list['point'] = point
open_grid = list(grid_list.point)
        
exemplar = Point(int(round(side/2)),int(round(side/2)))
im = Image.open(sfg.filename.loc[0])
im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
x = int(exemplar.x) * thumb_side
y = int(exemplar.y) * thumb_side
canvas.paste(im,(x,y))
open_grid.remove(exemplar)
        
for i in range(1,n):
    im = Image.open(sfg.filename.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    closest_open = min(open_grid,key=lambda x: exemplar.distance(x))
    x = int(closest_open.x) * thumb_side
    y = int(closest_open.y) * thumb_side
    canvas.paste(im,(x,y))
    open_grid.remove(closest_open)
canvas.save(DIR+var+"_"+str(thumb_side)+"_"+str(sample_size)+".png")        